# Compare mom6-global-01 spin up with ACCESS-OM2-01 RYF

We are finding that all the models (e.g. ACCESS-OM2-01 RYF, mom6-global-01, panan-005) have large drifts and biases at 37S, so would be problematic to use for forcing at the boundary of the panan. Instead: why not use a year from early on in one of these simulations when the bias is small. This notebook zooms into that early initial spinup period to choose which model is best and to assess drift.

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import matplotlib.path as mpath
import xarray as xr
import cmocean.cm as cmocean
import matplotlib.cm as cm
from collections import OrderedDict
import glob,os
import matplotlib.colors as col
from datetime import timedelta

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 125.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37003,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 125.20 GiB
Comm: tcp://127.0.0.1:41811,Total threads: 4
Dashboard: /proxy/35781/status,Memory: 17.89 GiB
Nanny: tcp://127.0.0.1:45333,


In [3]:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')

## Abyssal ocean temperature, at 37S, below 3000m

In [6]:
# ACCESS-OM2 thickness and area for depth averaging (not time varying):

temp_con = cc.querying.getvar('01deg_jra55v13_ryf9091','temp' , master_session,frequency='3 monthly', n=1)
temp_con = temp_con.sel(yt_ocean=-37,method='nearest')
# import edges of st_ocean and add lat/lon dimensions:
st_edges_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091', 'st_edges_ocean', master_session, n=1)
st_edges_array = st_edges_ocean.expand_dims({'xt_ocean':temp_con.xt_ocean}, axis=[1])

# adjust edges at bottom for partial thickness:
ht = cc.querying.getvar('01deg_jra55v13_ryf9091','ht' , master_session,n=1).sel(yt_ocean=-37,method='nearest')
st_edges_with_partial = st_edges_array.where(st_edges_array<ht, other=ht)
thickness = st_edges_with_partial.diff(dim='st_edges_ocean')

# change coordinate of thickness to st_ocean (needed for multipling with other variables):
st_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091', 'st_ocean', master_session, n=1)
thickness['st_edges_ocean'] = st_ocean.values
thickness = thickness.rename(({'st_edges_ocean':'st_ocean'}))

In [7]:
area_t = cc.querying.getvar('01deg_jra55v13_ryf9091','area_t',master_session, ncfile = 'ocean_grid.nc',n=-1)
area_t = area_t.sel(yt_ocean=-37,method='nearest')
volume = area_t*thickness.sel(st_ocean=slice(3000,6000))
volume = volume.load()
volume_sum = volume.sum()
volume_sum = volume_sum.load()

In [9]:
# ACCESS-OM2, 01deg_jra55v13_ryf9091

abyss_temp_ryf = cc.querying.getvar('01deg_jra55v13_ryf9091', 'temp', master_session, frequency='3 monthly',
                         start_time='1900-01-01',end_time='1950-01-01') - 273.15
abyss_temp_ryf = abyss_temp_ryf.sel(yt_ocean=-37,method='nearest').sel(st_ocean=slice(3000,6000)).isel(time=slice(0,12))

# volume average:
abyss_temp_ryf = ((abyss_temp_ryf*volume).sum('xt_ocean').sum('st_ocean')/volume_sum).compute()

2023-02-02 09:42:32,043 - distributed.nanny - WARNING - Restarting worker
Exception ignored in: <function CachingFileManager.__del__ at 0x14f1c9f9c310>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 248, in __del__
    self.close(needs_lock=False)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 232, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2624, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2587, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2028, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID


In [11]:
# mom6

abyss_temp_mom6_global = cc.querying.getvar('global-01-v2', 'thetao', master_session, frequency='1 monthly')
abyss_temp_mom6_global = abyss_temp_mom6_global.sel(yh=-37,method='nearest').sel(z_l=slice(3000,6000)).isel(time=slice(0,36))

volcello = cc.querying.getvar('global-01-v2', 'volcello', master_session,ncfile='%.ocean_month_z.nc')
volcello = volcello.sel(yh=-37,method='nearest').sel(z_l=slice(3000,6000)).groupby('time.year').mean('time')
volume_sum = volcello.isel(year=0).sum().load()

# volume average:
abyss_temp_mom6_global = ((abyss_temp_mom6_global*volcello).sum('xh').sum('z_l')).compute()/volume_sum

Exception ignored in: <function CachingFileManager.__del__ at 0x146f6a56a1f0>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 248, in __del__
    self.close(needs_lock=False)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 232, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2624, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2587, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2028, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID
Exception ignored in: <function CachingFileManager.__del__ at 0x146f6a56a1f0>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 248, in __del__
  

In [12]:
# panan

abyss_temp_panan = cc.querying.getvar('panant-01-zstar-v13', 'thetao', master_session, frequency='1 monthly')
abyss_temp_panan = abyss_temp_panan.sel(yh=-37,method='nearest').sel(z_l=slice(3000,6000)).isel(time=slice(0,36))

volcello = cc.querying.getvar('panant-01-zstar-v13', 'volcello', master_session,ncfile='%.ocean_month_z.nc')
volcello = volcello.sel(yh=-37,method='nearest').sel(z_l=slice(3000,6000)).groupby('time.year').mean('time')
volume_sum = volcello.isel(year=0).sum().compute()

# volume average:
abyss_temp_panan = ((abyss_temp_panan*volcello).sum('xh').sum('z_l')).compute()/volume_sum

Exception ignored in: <function CachingFileManager.__del__ at 0x146f6a56a1f0>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 248, in __del__
    self.close(needs_lock=False)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 232, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2624, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2587, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2028, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID
Exception ignored in: <function CachingFileManager.__del__ at 0x146f6a56a1f0>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 248, in __del__
  

In [13]:
# panan-hycom

abyss_temp_panan2 = cc.querying.getvar('panant-01-hycom1-v13', 'thetao', master_session, frequency='1 monthly')
abyss_temp_panan2 = abyss_temp_panan2.sel(yh=-37,method='nearest').sel(z_l=slice(3000,6000)).isel(time=slice(0,36))

volcello = cc.querying.getvar('panant-01-hycom1-v13', 'volcello', master_session,ncfile='%.ocean_month_z.nc')
volcello = volcello.sel(yh=-37,method='nearest').sel(z_l=slice(3000,6000)).groupby('time.year').mean('time')
volume_sum = volcello.isel(year=0).sum().compute()

# volume average:
abyss_temp_panan2 = ((abyss_temp_panan2*volcello).sum('xh').sum('z_l')).compute()/volume_sum

2023-02-02 10:29:08,786 - distributed.worker - WARNING - Compute Failed
Key:       open_dataset-2dcde6dc-01b4-4df4-870a-340ae831817d
Function:  execute_task
args:      ((<function apply at 0x146f81ce6dc0>, <function open_dataset at 0x146f6a538280>, ['/g/data/ik11/outputs/mom6-panan/panant-01-hycom1-v13/output002/19910701.ocean_month_z.nc'], (<class 'dict'>, [['engine', None], ['chunks', (<class 'dict'>, [['time', 1], ['z_l', 11], ['yh', 121], ['xh', 515]])]])))
kwargs:    {}
Exception: "RuntimeError('NetCDF: Not a valid ID')"



RuntimeError: NetCDF: Not a valid ID